<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/llm/airoboros_13b_gptq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install auto-gptq

Load model

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

model_name_or_path = "TheBloke/airoboros-13b-gpt4-GPTQ"
model_basename = "airoboros-13b-gpt4-GPTQ-4bit-128g.no-act.order"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

Inference with model generate

In [ ]:
print("\n\n*** Generate:")

prompt_template = """
USER: I want you to act as a programmer. I will provide the description of an image, you should output the corresponding layout of this image. Each object in the image is one rectangle or square box in the layout and size of boxes should be as large as possible comapre to the image size. The size of the image is 512 * 512 You should return each object and the corresponding coordinate of its boxes.
the prompt: \"three cats in the field\", 
ASSISTANT :
cat: (51, 82, 399, 279)
cat: (288, 128, 472, 299)
cat: (27, 355, 418, 494)

USER:
the prompt: \"a cat on the left of a dog on the road\"
ASSISTANT:
cat: (63, 196, 223, 394)
dog: (289, 131, 466, 360)

USER:
the prompt: \"four balls in the room\"
ASSISTANT:
ball: (72, 81, 254, 243)
ball: (316, 44, 483, 218)
ball: (287, 295, 453, 462)
ball: (50, 323, 196, 484)

USER:
the prompt: \"A donut to the right of a toilet\"
ASSISTANT:
donut: (287, 140, 467, 335)
toilet: (31, 97, 216, 286)
USER:
the prompt: \"A cat sitting on the top of a car\"
ASSISTANT:
car: (94, 236, 414, 407)
cat: (124, 139, 273, 252)

USER:
the prompt: \"A dog underneath a tree\"
ASSISTANT:
dog: (133, 232, 308, 445)
tree: (121, 29, 324, 258)

USER:
the prompt: \"a small ball is put on the top of a box on the table. there is a red vase on the right of the box on the table\"
ASSISTANT:
small ball: (92, 30, 165, 134)
box: (93, 132 , 205, 324, 310)
red vase: (214, 164, 297, 301)
table: (36, 259, 418, 463)
USER:
the prompt: \"a table at the center of room with four chairs. There is a sofa on the right side of the room\"

ASSISTANT:"""

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt_template):])

Text generation pipeline

In [ ]:
prompt = "Tell me about AI"
prompt_template=f'''USER: {prompt}
ASSISTANT:'''

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

print(pipe(prompt_template)[0]['generated_text'][len(prompt_template):])